In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision.transforms as transforms
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.cm as cm

KeyboardInterrupt: 

In [1]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NameError: name 'torch' is not defined

# Load the data

data/

│-- 2018/

│   │-- image_1.npy

│   │-- image_2.npy

│   └-- ...

│-- 2024/

│   │-- image_1.npy

│   │-- image_2.npy

│   └-- ...

│-- masks/  (zone de construction)

│   │-- mask_1.npy

│   │-- mask_2.npy

│   └-- ...

Get the data

In [ ]:
data_dir_2018 = "/path/to/2018/*.npy" # 64 x 64 x 12bandes (dont masque)
data_dir_2024 = "/path/to/2024/*.npy" # 64 x 64 x 11bandes

# à vérifier si c'est bien sous forme de liste :
#
# import glob
# data_dir_2018 = sorted(glob.glob("/path/to/2018/*.npy"))
# data_dir_2024 = sorted(glob.glob("/path/to/2024/*.npy"))

Prepare pytorch dataset

In [ ]:
class DatasetCadastre(torch.utils.data.Dataset):
    def __init__(self, data_dir_2018, data_dir_2024, transform=None):
        self.data_2018 = data_dir_2018
        self.data_2024 = data_dir_2024
        self.transform = transform

    def __getitem__(self, index):
        # Chargement des données
        img_2018 = np.load(self.data_2018[index]).astype(np.float32) # (64, 64, 12)
        img_2024 = np.load(self.data_2024[index]).astype(np.float32) # (64, 64, 11)

        # Ajustement de l'ordre pytorch (channels, size)
        img_2018 = np.transpose(img_2018, (2,0,1))
        img_2024 = np.transpose(img_2024, (2,0,1))

        # Transformation en tensor pytorch
        img_2018 = torch.tensor(img_2018)
        img_2024 = torch.tensor(img_2024)

        # Si transformation appliquée
        if self.transform:
          img_2018 = self.transform(img_2018)
          img_2024 = self.transform(img_2024)

        return img_2018, img_2024

    def __len__(self):
        return len(self.data_2018)

# Dataset gobal --> à voir si on ne fait pas plutôt un dataset pour chaque donnée (2018 et 2024) ?
cadastre_dataset = DatasetCadastre(data_dir_2018, data_dir_2024)


NameError: name 'torch' is not defined

Prepare dataloader

In [ ]:
batch_size = 512
num_threads = 4

train_loader = torch.utils.data.DataLoader(
    dataset=cadastre_dataset, batch_size=batch_size, shuffle=True, num_workers=num_threads
)
valid_loader = torch.utils.data.DataLoader(
    dataset=cadastre_dataset, batch_size=batch_size, shuffle=False, num_workers=num_threads
)

Normalize data

In [ ]:
# à voir car déjà normalisée ?

# Importe computed indices

In [ ]:
# Permet de calculer les indices sur les bandes prédites (les indices vrais étant déjà compris dans une des bandes des images)

import ...
NDVI_commputed = ...
...

# def NDVI_computed(img_tensor):
#     IR = img_tensor[...,:,:] # bande IR
#     RED = img_tensor[...,:,:] # bande RED
#     NDVI = (IR-RED)/(IR+RED)
#     return NDVI

# for img_2018, img_2024 in train_loader:
#     NDVI_2018 = NDVI_computed(img_2018)
#     NDVI_2024 = NDVI_computed(img_2024)
#     break

# Model definition

In [ ]:
class Conv2DRegressionModel(nn.Module): # prédiction de l'image 2024
    def __init__(self, int_channels: int = 11):
        super(Conv2DRegressionModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(int_channels, 32, kernel_size=3, padding=1),
            nn.LazyBatchNorm2d(),
            # nn.BatchNorm2d(int_channels*2)
            nn.ReLU(),
            nn.MaxPool2d(2, kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.LazyBatchNorm2d(),
            # nn.BatchNorm2d(int_channels*4)
            nn.ReLU(),
            nn.MaxPool2d(2, kernel_size=2),
            # nn.FullyConnected ?
            nn.Flatten(),
            nn.Linear(128*8*8, 512),
            nn.ReLU(),
            nn.Linear(512, 64*64)
        )

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        y = self.layers(X)
        y = y.view(-1, 64, 64) # sortie en image NDVI
        return y


# Train the model

In [ ]:
n_epochs = 10
learning_rate = 0.001

model = Conv2DRegressionModel()
model = model.to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_set_len = len(train_loader)
val_set_len = len(valid_loader)

train_loss_comp, val_loss_comp = [], []
train_loss_pred, val_loss_pred = [], []
train_loss_comp_pred, val_loss_comp_pred = [], []

for epoch in range(n_epochs):
    model.train()
    accu_comp = 0.0
    accu_pred = 0.0
    accu_comp_pred = 0.0

    for img_2018, img_2024 in train_loader:
        # img_2018, img_2024 = img_2018.to(device), img_2024.to(device)

        # Forward pass
        img_2024_pred = model(img_2018) # image 2024 prédite à partir de 2018

        # NDVI
        NDVI_2018 = img_2018[...,:,:] # NDVI 2018 calculé présent à la bande ... (vrai)
        NDVI_2024 = img_2024[...,:,:] # NDVI 2014 calculé présent à la bande ... (vrai)
        NDVI_2024_comp = NDVI_computed(img_2024_pred) # NDVI 2024 caclulé à partir des bandes prédites
        NDVI_2024_pred = img_2024_pred[...,:,:] # NDVI 2024 prédit par le réseau de neurones à la bande ...

        # NDMI
        NDMI_2018 = img_2018[...,:,:]
        NDMI_2024 = img_2024[...,:,:]
        NDMI_2024_comp = NDMI_computed(img_2024_pred)
        NDMI_2024_pred = img_2024_pred[...,:,:]

        # NDBI
        NDBI_2018 = img_2018[...,:,:]
        NDBI_2024 = img_2024[...,:,:]
        NDBI_2024_comp = NDBI_computed(img_2024_pred)
        NDBI_2024_pred = img_2024_pred[...,:,:]

        # NDWI
        NDWI_2018 = img_2018[...,:,:]
        NDWI_2024 = img_2024[...,:,:]
        NDWI_2024_comp = NDWI_computed(img_2024_pred)
        NDWI_2024_pred = img_2024_pred[...,:,:]

        # BSI
        BSI_2018 = img_2018[...,:,:]
        BSI_2024 = img_2024[...,:,:]
        BSI_2024_comp = BSI_computed(img_2024_pred)
        BSI_2024_pred = img_2024_pred[...,:,:]

        # Regroupement des indices
        Ind_2018 = [NDVI_2018, NDMI_2018, NDBI_2018, NDWI_2018, BSI_2018]
        Ind_2024 = [NDVI_2024, NDMI_2024, NDBI_2024, NDWI_2024, BSI_2024]
        Ind_2024_comp = [NDVI_2024_comp, NDMI_2024_comp, NDBI_2024_comp, NDWI_2024_comp, BSI_2024_comp]
        Ind_2024_pred = [NDVI_2024_pred, NDMI_2024_pred, NDBI_2024_pred, NDWI_2024_pred, BSI_2024_pred]

        weights = torch.tensor([0.4, 0.2, 0.2, 0.1, 0.1], device=device) # pondération à ajuster
        loss_comp = sum(w * loss_fn(comp, true) for w, comp, true in zip(weights, Ind_2024_comp, Ind_2024)) # comparaison des indices calculés
        loss_pred = sum(w * loss_fn(pred, true) for w, pred, true in zip(weights, Ind_2024_pred, Ind_2024)) # comparaison des indices prédits
        loss_comp_pred = sum(w * loss_fn(comp, pred) for w, comp, pred in zip(weights, Ind_2024_comp, Ind_2024_pred)) # comparaison des indices calculés et prédits
        total_loss = loss_comp + loss_pred + loss_comp_pred

        accu_comp += loss_comp.item()
        accu_pred += loss_pred.item()
        accu_comp_pred += loss_comp_pred.item()

        # Backward pass
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
    train_loss_comp.append(accu_comp / train_set_len)
    train_loss_pred.append(accu_pred / train_set_len)
    train_loss_comp_pred.append(accu_comp_pred / train_set_len)

    # Validation - no gradient & eval mode
    model.eval()
    accu_comp = 0.0
    accu_pred = 0.0
    accu_comp_pred = 0.0
    with torch.no_grad():
        for img_2018, img_2024 in valid_loader:
            # img_2018, img_2024 = img_2018.to(device), img_2024.to(device)

            # Forward pass
            img_2024_pred = model(img_2018) # image 2024 prédite à partir de 2018

            # NDVI
            NDVI_2018 = img_2018[...,:,:] # NDVI 2018 calculé présent à la bande ... (vrai)
            NDVI_2024 = img_2024[...,:,:] # NDVI 2014 calculé présent à la bande ... (vrai)
            NDVI_2024_comp = NDVI_computed(img_2024_pred) # NDVI 2024 caclulé à partir des bandes prédites
            NDVI_2024_pred = img_2024_pred[...,:,:] # NDVI 2024 prédit par le réseau de neurones à la bande ...

            # NDMI
            NDMI_2018 = img_2018[...,:,:]
            NDMI_2024 = img_2024[...,:,:]
            NDMI_2024_comp = NDMI_computed(img_2024_pred)
            NDMI_2024_pred = img_2024_pred[...,:,:]

            # NDBI
            NDBI_2018 = img_2018[...,:,:]
            NDBI_2024 = img_2024[...,:,:]
            NDBI_2024_comp = NDBI_computed(img_2024_pred)
            NDBI_2024_pred = img_2024_pred[...,:,:]

            # NDWI
            NDWI_2018 = img_2018[...,:,:]
            NDWI_2024 = img_2024[...,:,:]
            NDWI_2024_comp = NDWI_computed(img_2024_pred)
            NDWI_2024_pred = img_2024_pred[...,:,:]

            # BSI
            BSI_2018 = img_2018[...,:,:]
            BSI_2024 = img_2024[...,:,:]
            BSI_2024_comp = BSI_computed(img_2024_pred)
            BSI_2024_pred = img_2024_pred[...,:,:]

            # Regroupement des indices
            Ind_2018 = [NDVI_2018, NDMI_2018, NDBI_2018, NDWI_2018, BSI_2018]
            Ind_2024 = [NDVI_2024, NDMI_2024, NDBI_2024, NDWI_2024, BSI_2024]
            Ind_2024_comp = [NDVI_2024_comp, NDMI_2024_comp, NDBI_2024_comp, NDWI_2024_comp, BSI_2024_comp]
            Ind_2024_pred = [NDVI_2024_pred, NDMI_2024_pred, NDBI_2024_pred, NDWI_2024_pred, BSI_2024_pred]

            weights = torch.tensor([0.4, 0.2, 0.2, 0.1, 0.1], device=device) # pondération à ajuster
            loss_comp = sum(w * loss_fn(comp, true) for w, comp, true in zip(weights, Ind_2024_comp, Ind_2024)) # comparaison des indices calculés
            loss_pred = sum(w * loss_fn(pred, true) for w, pred, true in zip(weights, Ind_2024_pred, Ind_2024)) # comparaison des indices prédits
            loss_comp_pred = sum(w * loss_fn(comp, pred) for w, comp, pred in zip(weights, Ind_2024_comp, Ind_2024_pred)) # comparaison des indices calculés et prédits

            accu_comp += loss_comp.item()
            accu_pred += loss_pred.item()
            accu_comp_pred += loss_comp_pred.item()
        val_loss_comp.append(accu_comp / val_set_len)
        val_loss_pred.append(accu_pred / val_set_len)
        val_loss_comp_pred.append(accu_comp_pred / val_set_len)

         # Save the best models
        if val_loss_comp[-1] == min(val_loss_comp):
            torch.save(model.state_dict(), "save_model_comp.pt")
            print("Model 'comp' saved for epoch", epoch)
        if val_loss_pred[-1] == min(val_loss_pred):
            torch.save(model.state_dict(), "save_model_pred.pt")
            print("Model 'pred' saved for epoch", epoch)
        if val_loss_comp_pred[-1] == min(val_loss_comp_pred):
            torch.save(model.state_dict(), "save_model_comp_pred.pt")
            print("Model 'comp_pred' saved for epoch", epoch)

    print("Train loss 'comp'", train_loss_comp[-1])
    print("Val loss 'comp'", val_loss_comp[-1])

    print("Train loss 'pred'", train_loss_pred[-1])
    print("Val loss 'pred'", val_loss_pred[-1])

    print("Train loss 'comp_pred'", train_loss_comp_pred[-1])
    print("Val loss 'comp_pred'", val_loss_comp_pred[-1])


In [ ]:
plt.plot(train_loss_comp, label="Train Loss 'comp'")
plt.plot(val_loss_comp, label="Validation Loss 'comp'")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

plt.plot(train_loss_pred, label="Train Loss 'pred'")
plt.plot(val_loss_pred, label="Validation Loss 'pred'")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

plt.plot(train_loss_comp_pred, label="Train Loss 'comp_pred'")
plt.plot(val_loss_comp_pred, label="Validation Loss 'comp_pred'")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()